# Example: VQE program

This tutorial will be demonstation of creating VQE Program as well as migration guide on how you can replicate IBM Quantum VQE custom runtime program.

Let's first get information on what is VQE runtime program and what inputs and outputs it has.

**Description** of runtime program is: Variational Quantum Eigensolver (VQE) to find the minimal eigenvalue of a Hamiltonian.

**Inputs**:

| name | type | description |
| ---- | ---- | ----------- |
| ansatz | object | A parameterized quantum circuit preparing the ansatz wavefunction for the VQE. It is assumed that all qubits are initially in the 0 state. | 
|aux_operators|array| A list or dict (with strings as keys) of operators of type PauliSumOp to be evaluated at the final, optimized state.|
|initial_layout | [null,array,object] | Initial position of virtual qubits on the physical qubits of the quantum device. Default is None. |
| initial_parameters|[array,string]|Initial parameters of the ansatz. Can be an array or the string ``'random'`` to choose random initial parameters.|
|max_evals_grouped|integer|The maximum number of parameter sets that can be evaluated at once. Defaults to the minimum of 2 times the number of parameters, or 1000.|
|measurement_error_mitigation|boolean|Whether to apply measurement error mitigation in form of a complete measurement fitter to the measurements. Defaults to False.|
|operator|object|The Hamiltonian whose smallest eigenvalue we're trying to find. Should be PauliSumOp|
|optimizer|object|The classical optimizer used in to update the parameters in each iteration. Can be either any of Qiskit's Optimizer classes. If a dictionary, only SPSA and QN-SPSA are supported and the dictionary must specify the name and options of the optimizer, e.g. ``{'name': 'SPSA', 'maxiter': 100}``.|
|shots|integer|The number of shots used for each circuit evaluation. Defaults to 1024.|

**Return values**

| name | type | description |
| ---- | ---- | ----------- |
|aux_operator_eigenvalues|array|The expectation values of the auxiliary operators at the optimal state. This is a numpy array.|
|cost_function_evals|integer|The number of cost function (energy) evaluations.|
|eigenstate|object|The square root of sampling probabilities for each computational basis state of the circuit with optimal parameters.|
|eigenvalue|array|The estimated eigenvalue. This is a complex number.|
|optimal_parameters|null|Not supported at the moment, therefore ``None``.|
|optimal_point|array|The optimal parameter values found during the optimization. This is a numpy array.|
|optimal_value|number|The smallest value found during the optimization. Equal to the ``eigenvalue`` attribute. This is a float.|
|optimizer_evals|integer|The number of steps of the optimizer.|
|optimizer_history|object|A dictionary containing information about the function evaluations (not necessarily the actual parameter value!): the current evaluation count, the parameters, the energy and the standard deviation.|
|optimizer_time|number|The total time taken by the optimizer. This is a float.|

We will also add optional `QiskitRuntimeService` as an argument to use that to access real devices.


With that information we can start drafting our program implementation in `vqe.py` file.

What our program should do:
1. parse input arguments
2. create run_vqe function that accepts estimator instance, creates VQE and runs calculation
3. decide which estimator to use and run vqe
    - if runtime service was passed then create a session and run `run_vqe` function
    - if runtime service was not passed then use stantard qiskit estimator
4. save results from vqe

Roughly our VQE program will look like this

```python
# vqe.py

import ...

def run_vqe(
    estimator: BaseEstimator,
    ansatz: QuantumCircuit,
    operator: PauliSumOp,
    optimizer: Optimizer,
    initial_parameters: Optional[np.ndarray] = None
):
    vqe = VQE(
        estimator,
        ansatz,
        optimizer,
        initial_point=initial_parameters
    )
    return vqe.compute_minimum_eigenvalue(operator)


arguments = get_arguments()

service = arguments.get("service")
ansatz = arguments.get("ansatz")
operator = arguments.get("operator")
initial_parameters = arguments.get("initial_parameters") 
optimizer = ...

...

if service is not None:
    # if we have service we need to open a session and create estimator
    backend = arguments.get("backend", "ibmq_qasm_simulator")
    with Session(service=service, backend=backend) as session:
        estimator = Estimator(session=session, options=options) # qiskit_ibm_runtime.Estimator
        vqe_result = run_vqe( estimator=estimator, ...)
else:
    # if we do not have a service let's use standart local estimator
    estimator = QiskitEstimator() # qiskit.primitives.Estimator
    vqe_result = run_vqe(estimator=estimator, ...)

save_result({
    "cost_function_evals": vqe_result.cost_function_evals,
    "eigenvalue": vqe_result.eigenvalue,
    "optimal_point": vqe_result.optimal_point.tolist(),
    "optimal_value": vqe_result.optimal_value,
    "optimizer_evals": vqe_result.optimizer_evals,
    "optimizer_time": vqe_result.optimizer_time,
    ...
})

```

At this point we have our program implemented. Now we need to actually run it. But before let's prepare input arguments from our VQE program.

In [2]:
import numpy as np

from qiskit.circuit.library import RealAmplitudes

from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import QubitConverter, ParityMapper
from qiskit_ibm_runtime import QiskitRuntimeService, Estimator, Session, Options

USE_RUNTIME_SERVICE = False

ansatz = RealAmplitudes(num_qubits=4, reps=2)
initial_point = np.random.uniform(-np.pi, np.pi, 12)
driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.72"  # Two Hydrogen atoms, 0.72 Angstrom apart
)
molecule = driver.run()
qubit_converter = QubitConverter(ParityMapper())
hamiltonian = qubit_converter.convert(molecule.second_q_ops()[0])

service = None
if USE_RUNTIME_SERVICE:
    service = QiskitRuntimeService()
    backend = "ibmq_qasm_simulator"

input_arguments = {
    "ansatz": RealAmplitudes(num_qubits=4, reps=2),
    "aux_operators": None,
    "initial_layout": None,
    "initial_parameters": initial_point,
    "max_evals_grouped": None,
    "measurement_error_mitigation": None,
    "operator": hamiltonian,
    "optimizer": "spsa",
    "shots": None,
    "service": service
}

input_arguments

{'ansatz': <qiskit.circuit.library.n_local.real_amplitudes.RealAmplitudes at 0x7fe570ac5b10>,
 'aux_operators': None,
 'initial_layout': None,
 'initial_parameters': array([ 1.96607313,  0.6418064 ,  2.83542552,  2.07848803, -2.14744829,
         2.94371138, -2.27965871, -2.74576187, -2.38276942,  1.4102545 ,
        -2.74783675, -2.81548099]),
 'max_evals_grouped': None,
 'measurement_error_mitigation': None,
 'operator': PauliSumOp(SparsePauliOp(['IIII', 'IIIZ', 'IIZZ', 'IIZI', 'IZZI', 'IZZZ', 'ZXIX', 'IXZX', 'ZXZX', 'IXIX', 'ZZII', 'ZZIZ', 'IZIZ', 'ZZZZ', 'ZIZI'],
               coeffs=[-0.8054406 +0.j,  0.17452525+0.j, -0.23287429+0.j,  0.12177673+0.j,
   0.17452525+0.j,  0.1696433 +0.j,  0.04502464+0.j, -0.04502464+0.j,
  -0.04502464+0.j,  0.04502464+0.j, -0.23287429+0.j,  0.16680137+0.j,
   0.16680137+0.j,  0.17533917+0.j,  0.12177673+0.j]), coeff=1.0),
 'optimizer': 'spsa',
 'shots': None,
 'service': None}

With arguments prepared we can create our quantum serverless client, setup provider and run our program

In [3]:
from quantum_serverless import QuantumServerless, Provider
import os

In [4]:
provider = Provider(
    username="user",
    password="password123",
    host=os.environ.get("GATEWAY_HOST", "http://localhost:8000"),
)

serverless = QuantumServerless(provider)
serverless

<QuantumServerless | providers [gateway-provider]>

In [5]:
from quantum_serverless import Program

program = Program(
    title="VQE",
    entrypoint="vqe.py",
    working_dir="./source_files/vqe/"
)

job = serverless.run(program, arguments=input_arguments)
job

<Job | 95e03817-663a-42fa-b199-0ee5016d2d49>

In [6]:
job.status()

'SUCCEEDED'

In [7]:
job.logs().split("\n")[-10:-1]

['243: -1.252362175485805',
 '244: -1.245728828896075',
 '245: -1.2505250649172739',
 '246: -1.2572001670623152',
 '247: -1.2611976192184255',
 '248: -1.258139706050951',
 '249: -1.2322759435538633',
 '250: -1.224270224547061',
 '251: -1.2645989534506294']

In [8]:
job.result()

{'aux_operator_eigenvalues': [],
 'cost_function_evals': 200,
 'eigenstate': None,
 'eigenvalue': -1.2645989534506294,
 'optimal_parameters': None,
 'optimal_point': [3.0276372310610675,
  0.13553102047367085,
  2.5087090109567773,
  1.8492042571718177,
  -2.222021041944339,
  3.5428224235643113,
  -1.2788913910370245,
  -2.8246845124804763,
  -2.728535775675822,
  3.4434351915865857,
  -1.9083062252923417,
  -1.7416966890283174],
 'optimal_value': -1.2645989534506294,
 'optimizer_evals': None,
 'optimizer_history': {},
 'optimizer_time': 0.8049108982086182}